In [1]:
!pip install psycopg2

     ---------------------------------------- 1.2/1.2 MB 2.4 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import psycopg2

In [8]:
try: 
    conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=password")
except psycopg2.Error as e:
    print("Error: Unable to connect to postgres database")
    print(e)
else:
    print("connected!")

connected!


In [9]:
try: 
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get cursor to db")
    print(e)
else:
    print("cursor created!")

cursor created!


In [10]:
conn.set_session(autocommit=True)

In [24]:
try:
    cur.execute("CREATE DATABASE f_datamodel_etl")
except psycopg2.Error as e:
    print(e)

database "f_datamodel_etl" already exists



In [33]:
conn.close()

try: 
    conn = psycopg2.connect("host=localhost dbname=f_datamodel_etl user=postgres password=password")
except psycopg2.Error as e:
    print("Error: Unable to connect to postgres database")
    print(e)
else:
    print("connected!")
    

try: 
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get cursor to db")
    print(e)
else:
    print("cursor created!")

connected!
cursor created!


In [34]:
conn.set_session(autocommit=True)

In [35]:
try:
    cur.execute("CREATE TABLE public.Insta_Influencer (username varchar(100) not null, name varchar(300) not null, followers varchar(50) not null, audience_country varchar(50) not null, \
    authentic_engagement varchar(50) not null, engagement_avg varchar(50) not null, category_1 varchar(100) null, category_2 varchar(100) null);")
except psycopg2.Error as e:
    print("Error: failure to create table")
    print(e)
else:
    print("Table created")

Table created


In [37]:
try:
    cur.execute("CREATE TABLE public.insta_influencer_June ( username varchar(100) NOT NULL, name varchar(300) NOT NULL, \
	subscribers_count varchar(50) NOT NULL, \
	audience_country varchar(50) NOT NULL, \
	likes_avg varchar(50) NOT NULL, \
	comments_avg varchar(50) NOT NULL, \
	category_1 varchar(100) NULL, \
	category_2 varchar(100) NULL \
);")
except psycopg2.Error as e:
    print("Error: failure to create table")
    print(e)
else:
    print("Table created")

Table created


In [39]:
try:
    cur.execute("CREATE TABLE public.insta_influencer_Sep ( \
	sr_no int not null, \
	username varchar(100) NOT NULL, \
	name varchar(300) NOT NULL, \
	subscribers varchar(50) NOT NULL, \
	audience_country varchar(50) NOT NULL, \
	likes_avg varchar(50) NOT NULL, \
	authentic_engagement varchar(50) not null, \
	engagement_avg varchar(50) NOT NULL, \
	category_1 varchar(100) NULL, \
	category_2 varchar(100) NULL \
);")
except psycopg2.Error as e:
    print("Error: failure to create table")
    print(e)
else:
    print("Table created")

Table created


In [44]:
import pandas as pd

In [45]:
df = pd.read_csv(r'C:\Umair_Workspace\Development\Projects\1_DataModel_ETL\Dataset\Social Media Influencers in 2022\toProcess\social media influencers - instagram.csv')
print(df)

    Influencer insta name instagram name                 category_1  \
0                     433            433         Sports with a ball   
1            __youngbae__        TAEYANG                      Music   
2             _agentgirl_  НАСТЯ ИВЛЕЕВА                      Shows   
3             _imyour_joy            Joy                  Lifestyle   
4           _jeongjaehyun        Jaehyun                        NaN   
..                    ...            ...                        ...   
995               zendaya        Zendaya  Cinema & Actors/actresses   
996                zidane         zidane         Sports with a ball   
997                zkdlin            KAI                      Music   
998    zoeisabellakravitz   Zoë Kravitz  Cinema & Actors/actresses   
999               zoesugg      Zoë Sugg                  Lifestyle   

             category_2 Followers Audience country(mostly)  \
0                   NaN     48.5M                    Spain   
1                   NaN

In [56]:
df.columns = [c.replace(' ','_') for c in df.columns]

In [60]:
df = df.rename(columns = {'Audience_country(mostly)' : 'Audience_country'})

In [66]:
df = df.rename(columns = {'Authentic_engagement\r\n' : 'Authentic_engagement'})

In [68]:
df = df.rename(columns = {'Engagement_avg\r\n' : 'Engagement_avg'})

In [65]:
print(df.columns)

Index(['Influencer_insta_name', 'instagram_name', 'category_1', 'category_2',
       'Followers', 'Audience_country', 'Authentic_engagement\r\n',
       'Engagement_avg\r\n'],
      dtype='object')


In [71]:
try:
    for index,row in df.iterrows():
        cur.execute("INSERT INTO public.insta_influencer \
        (username, name, followers, audience_country, authentic_engagement, engagement_avg, category_1, category_2) \
        VALUES(%s, %s, %s, %s, %s, %s, %s, %s)", (row.Influencer_insta_name, row.instagram_name, row.Followers, row.Audience_country, row.Authentic_engagement, row.Engagement_avg, row.category_1, row.category_2))
except psycopg2.Error as e:
    print("Error: unable to insert data")
    print(e)

In [74]:
try:
    cur.execute("select * from public.insta_influencer limit 10")
except psycopg2.Error as e:
    print("Error: error select")
    print(e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

('433', '433', '48.5M', 'Spain', '383.1K', '637K', 'Sports with a ball', 'NaN')
('__youngbae__', 'TAEYANG', '12.7M', 'Indonesia', '478K', '542.3K', 'Music', 'NaN')
('_agentgirl_', 'НАСТЯ ИВЛЕЕВА', '18.8M', 'Russia', '310.8K', '377.9K', 'Shows', 'NaN')
('_imyour_joy', 'Joy', '13.5M', 'Indonesia', '1.1M', '1.4M', 'Lifestyle', 'NaN')
('_jeongjaehyun', 'Jaehyun', '11.1M', 'Indonesia', '2.5M', '3.1M', 'NaN', 'NaN')
('_mariahwasa', 'HWASA', '7.9M', 'Brazil', '915.1K', '1.2M', 'NaN', 'NaN')
('_rl9', 'Robert Lewandowski', '25M', 'Poland', '588.6K', '749K', 'Sports with a ball', 'NaN')
('_seorina', '설인아 sᴇᴏʀɪɴᴀ', '3M', 'South Korea', '803.9K', '997.8K', 'Lifestyle', 'Cinema & Actors/actresses')
('_shotaroo_', 'SHOTARO ショウタロウ', '4.9M', 'Indonesia', '860.1K', '1.1M', 'NaN', 'NaN')
('03_hu', '박지후', '4.1M', 'South Korea', '711K', '872.6K', 'NaN', 'NaN')
